# Exploratory analysis of the dataset

Banter and more banter

https://www.kaggle.com/notgibs/500-greatest-albums-of-all-time-rolling-stone?select=albumlist.csv

country_codes= ['AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'EC', 'SV', 'EE', 'FI', 'FR', 'DE', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'ID', 'IE', 'IT', 'JP', 'LV', 'LI', 'LT', 'LU', 'MY', 'MT', 'MX', 'MC', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'ES', 'SK', 'SE', 'CH', 'TW', 'TR', 'GB', 'US', 'UY']

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
from enrich_functions import *
from analysis_functions import *
import re

## Data cleaning

In [2]:
top500_raw = pd.read_csv("../dataset/albumlist.csv", index_col=0, encoding = "ISO-8859-1")
top500_raw.head()

,Year,Album,Artist,Genre,Subgenre
Number,,,,,
1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock


In [3]:
top500_raw.columns = map(str.lower, top500_raw.columns)

Separate generes and subgeneres in different columns.

In [4]:
clean = lambda x: x.replace("Ê"," ").split(", ")
top500_raw["genre"]=top500_raw["genre"].apply(clean)
top500_raw["subgenre"]=top500_raw["subgenre"].apply(clean)

Renaming wrong album names and dropping the ones that arent in spoty

In [5]:
def wrong(x):
    dic={'The Beatles ("The White Album")':"The White Album",
          'Sign "Peace" the Times':'Sign "O" the Times',
          'The Band ("The Brown Album")':'The Band',
          'Back to Mono (1958-1969)':'Back To Mono',
          '[Led Zeppelin IV]':'Led Zeppelin IV',
          'Saturday Night Fever: The Original Movie Sound Track':'Saturday Night Fever',
          "The B 52's / Play Loud":"The B 52's",
          "The Very Best of Linda Ronstadt":"The Best of Linda Ronstadt",
          "The Anthology: 1961-1977": "The Very Best of Curtis Mayfield",
          'Blues Breakers With Eric Clapton ("The Beano Album")':"Bluesbreakers",
          "Bo Diddley / Go Bo Diddley":"Go Bo Diddley",
          "Can't Buy a Thrill":"Cant Buy a Thrill",
          'Metallica ("The Black Album")':"Metallica",
          "Trans Europa Express":"Trans-Europe Express",
          "'":"",
          "Honky Chteau":"Honky Chateau",
          "Cheap Trick at Budokan":"At Budokan"
          }
    xr=x
    for wrong,right in dic.items():
        xr=xr.replace(wrong,right)
    return xr
#wrong = lambda x: x.replace(wrong, right) for wrong, right in dict_rep.items()
top500_raw["album"]=top500_raw["album"].apply(wrong)


In [6]:
top500_raw.drop([178,196,198,214,245,278,289,346,363,377,400,421,449,170,459],inplace=True)

Enrich spoty

In [7]:
load_dotenv()
cid = os.getenv("SPOTIFY_ID")
secret = os.getenv("SPOTIFY_SECRET")
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
spotify

In [8]:
features_median = lambda x: get_album_features_median(spotify,x)


In [10]:
top500_raw["features"]=top500_raw["album"].apply(features_median)

In [11]:
top500_raw=pd.concat([top500_raw, top500_raw["features"].apply(pd.Series)], axis=1, sort=False)

get_wiki_views(lang,page,ini_year,ini_month,ini_day,end_year,end_month,end_day)

In [12]:
wiki= lambda x: get_wiki_views("es",x,2019,10,1,2020,10,1)
top500_raw["wiki_views_album"]=top500_raw["album"].apply(wiki)
top500_raw["wiki_views_artist"]=top500_raw["artist"].apply(wiki)

/home/ordovas/anaconda3/envs/ironhack/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [13]:
top500_raw.to_csv(r'../dataset/albumlist_clean.csv')

,year,album,artist,genre,subgenre,features,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,wiki_views_album,wiki_views_artist
Number,,,,,,,,,,,,,,,,,
2,1966,Pet Sounds,The Beach Boys,[Rock],"[Pop Rock, Psychedelic Rock]","{'danceability': 0.43, 'energy': 0.434, 'loudn...",0.4300,0.434,-7.9650,0.03350,0.5130,0.000008,0.1420,0.3640,116.6680,2281.0,10595.5
3,1966,Revolver,The Beatles,[Rock],"[Psychedelic Rock, Pop Rock]","{'danceability': 0.538, 'energy': 0.5609999999...",0.5380,0.561,-8.1825,0.03860,0.1825,0.000007,0.1355,0.6335,130.0410,222.0,91096.5
4,1965,Highway 61 Revisited,Bob Dylan,[Rock],"[Folk Rock, Blues Rock]","{'danceability': 0.483, 'energy': 0.691, 'loud...",0.4830,0.691,-6.9410,0.03200,0.7020,0.000000,0.2620,0.5910,99.5000,1004.0,29581.5
5,1965,Rubber Soul,The Beatles,"[Rock, Pop]",[Pop Rock],"{'danceability': 0.563, 'energy': 0.567, 'loud...",0.5630,0.567,-9.2730,0.03790,0.0371,0.000000,0.0943,0.6720,121.4020,4294.5,91096.5
7,1972,Exile on Main St.,The Rolling Stones,[Rock],"[Blues Rock, Rock & Roll, Classic Rock]","{'danceability': 0.518, 'energy': 0.734, 'loud...",0.5180,0.734,-5.7115,0.04145,0.5010,0.311000,0.2720,0.6205,114.2220,2122.5,38884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1972,Give It Up,Bonnie Raitt,[Rock],[Blues Rock],"{'danceability': 0.4575, 'energy': 0.58, 'loud...",0.4575,0.580,-7.3500,0.03120,0.4115,0.001240,0.1725,0.4845,119.6340,291.5,930.5
496,1969,Boz Scaggs,Boz Scaggs,[Rock],[Pop Rock],"{'danceability': 0.593, 'energy': 0.524, 'loud...",0.5930,0.524,-9.8450,0.04250,0.3570,0.000783,0.1250,0.4380,121.0810,883.5,883.5
497,2001,White Blood Cells,The White Stripes,[Rock],"[Indie Rock, Alternative Rock, Blues Rock, Gar...","{'danceability': 0.439, 'energy': 0.619, 'loud...",0.4390,0.619,-4.7070,0.04450,0.4540,0.000073,0.1110,0.3800,95.8850,176.0,6333.5
